# Imports

In [37]:
import numpy as np
import scipy as sp
import matplotlib.pyplot as plt

from time import perf_counter
from datetime import datetime

In [38]:
# Versions
import qiskit
import qiskit_algorithms

print(f"Qiskit version {qiskit.__version__}")
print(f"Qiskit algorithms version {qiskit_algorithms.__version__}")


# Quantum Circuits!
from qiskit import QuantumCircuit
from qiskit import QuantumRegister, ClassicalRegister


# General Imports
from qiskit.primitives import StatevectorSampler, StatevectorEstimator
from qiskit.quantum_info import Statevector, Operator, SparsePauliOp
from qiskit.visualization import plot_histogram, plot_bloch_multivector
from qiskit.transpiler.preset_passmanagers import generate_preset_pass_manager

# Noisy simulations
from qiskit_aer import AerSimulator
from qiskit_aer.noise import (
    NoiseModel,
    QuantumError,
    ReadoutError,
    depolarizing_error,
    pauli_error,
    thermal_relaxation_error,
)

# Running on a real quantum computer
from qiskit_ibm_runtime import QiskitRuntimeService
from qiskit_ibm_runtime import SamplerV2

# Run time-evolution problems
from qiskit_algorithms.time_evolvers import TrotterQRTE, TimeEvolutionProblem


Qiskit version 1.3.2
Qiskit algorithms version 0.3.1


# General

## States and Bloch Vectors

In [ ]:
# Generate a random state
def rstate(unitary: bool = False) -> np.ndarray:
    # Chose the secret state for Alice
    theta = np.random.random() * np.pi
    phi = np.random.random() * (2 * np.pi)

    # Compute the coefficients
    a = np.cos(0.5 * theta)
    b = np.sin(0.5 * theta) * np.exp(1j * phi)

    if unitary:
        # Unitary matrix that makes that state
        return np.array([[a, -np.conj(b)], [b, np.conj(a)]])
    else:
        return np.array([a, b])

In [40]:
# Reconstruct the state from the Bloch vector
def state_from_bloch(bloch: list) -> list:
    # Get angles
    theta = np.arccos(bloch[2])
    phi = np.arctan2(bloch[1], bloch[0])

    # Get the state
    a = np.cos(0.5 * theta)
    b = np.sin(0.5 * theta) * np.exp(1j * phi)

    return [a, b]

In [41]:
def bloch_from_state(state: list) -> list:
    # Get phases
    phi0 = np.angle(state[0])
    phi1 = np.angle(state[1])

    # Get angles
    theta = 2 * np.arccos(np.abs(state[0]))
    phi = phi1 - phi0

    return [np.sin(theta)*np.cos(phi), np.sin(theta)*np.sin(phi), np.cos(theta)]

In [ ]:
# Get the density matrix for some Bloch vector
def density_from_bloch(bloch: list | np.ndarray) -> np.ndarray:
    # Pauli matrices
    I = np.eye(2)
    X = np.array([[0, 1], [1, 0]])
    Y = np.array([[0, -1j], [1j, 0]])
    Z = np.array([[1, 0], [0, -1]])
    
    # Construct the density matrix
    rho = 0.5 * (I + bloch[0]*X + bloch[1]*Y + bloch[2]*Z)
    return rho

## Pauli Matrices

In [ ]:
# Calculate the pauli inverse matrix that we can apply on any matrix to get the coefficients of its Pauli expantion
def pauliinv(n: int) -> tuple[list, list, np.ndarray]:

    # Make sure n is a power of 2
    if not np.allclose(int(np.log2(n)), np.log2(n)):
        raise ValueError("The parameter n must be a power of 2.")

    # Define Pauli matrices
    si = np.array([[1, 0], [0, 1]])
    sx = np.array([[0, 1], [1, 0]])
    sy = np.array([[0, -1j], [1j, 0]])
    sz = np.array([[1, 0], [0, -1]])
    pp2 = [si, sx, sy, sz]
    ll2 = ["I", "X", "Y", "Z"]

    # Compute the higher-order pauli matrices and labels
    ppn = list(np.copy(pp2))
    lln = list(np.copy(ll2))
    for _ in range(int(np.log2(n)) - 1):
        ppn = [np.kron(s1, s2) for s1 in ppn for s2 in pp2]
        lln = [s1 + s2 for s1 in lln for s2 in ll2]
    
    # Compute the coefficients matrix for the equation a0 * si + a1 * sx + a2 * sy + a3 * sz = M where M is a matrix
    coeffn = np.array([list(ss.flatten()) for ss in ppn], dtype=np.complex128).T
    paulin = np.linalg.inv(coeffn)

    return lln, ppn, paulin


# Testing
ll2, pp2, pauli2 = pauliinv(2)
ll4, pp4, pauli4 = pauliinv(4)
ll8, pp8, pauli8 = pauliinv(8)

test2 = np.array([
    [-1 + 1j, 2],
    [2, 5]
])
test2_coeff = pauli2 @ test2.flatten()
test2_pauli = sum([ss * co for (ss, co) in zip(pp2, test2_coeff)])
print(f"Test for 2 x 2 matrix: {'Success!' if np.allclose(test2, test2_pauli) else 'Fail!'}")

test4 = np.array([
    [0.5,0,0,1j],
    [0,1.5,0,1],
    [0,0,2.5,0],
    [0,2,0,3.5]
])
test4_coeff = pauli4 @ test4.flatten()
test4_pauli = sum([ss * co for (ss, co) in zip(pp4, test4_coeff)])
print(f"Test for 4 x 4 matrix: {'Success!' if np.allclose(test4, test4_pauli) else 'Fail!'}")

g0 = 1 + 1j
test8 = np.array([
        [0.5,0,0,0,g0,g0,g0,g0],
        [0,1.5,0,0,g0,g0,g0,g0],
        [0,0,2.5,0,g0,g0,g0,g0],
        [0,0,0,3.5,g0,g0,g0,g0],
        [g0,g0,g0,g0,0.5,0,0,0],
        [g0,g0,g0,g0,0,1.5,0,0],
        [g0,g0,g0,g0,0,0,2.5,0],
        [g0,g0,g0,g0,0,0,0,3.5]  
])
test8_coeff = pauli8 @ test8.flatten()
test8_pauli = sum([ss * co for (ss, co) in zip(pp8, test8_coeff)])
print(f"Test for 8 x 8 matrix: {'Success!' if np.allclose(test8, test8_pauli) else 'Fail!'}")

Test for 2 x 2 matrix: Success!
Test for 4 x 4 matrix: Success!
Test for 8 x 8 matrix: Success!


# Running 

In [43]:
"""
# Initiate connection
TOKEN = "11cf40c4938ae90a81c46fcb5fba785e229c640a57f2b23d8e836f37f3de2d7abc2bc176c0742bbb3e85cf937774a89db0630913240faf5e82b2d63e9654ff16"
service = QiskitRuntimeService(channel="ibm_quantum", token=TOKEN)

# Choose a real quantum computer, which is not busy
backend = service.least_busy(operational=True, simulator=False)
print(f"{backend.name =}, {backend.num_qubits = }")

# Get noise profile
noise_real = NoiseModel.from_backend(backend)
"""

'\n# Initiate connection\nTOKEN = "11cf40c4938ae90a81c46fcb5fba785e229c640a57f2b23d8e836f37f3de2d7abc2bc176c0742bbb3e85cf937774a89db0630913240faf5e82b2d63e9654ff16"\nservice = QiskitRuntimeService(channel="ibm_quantum", token=TOKEN)\n\n# Choose a real quantum computer, which is not busy\nbackend = service.least_busy(operational=True, simulator=False)\nprint(f"{backend.name =}, {backend.num_qubits = }")\n\n# Get noise profile\nnoise_real = NoiseModel.from_backend(backend)\n'

In [44]:
def aersim(qc: QuantumCircuit, shots: int = 1024, noise: None | NoiseModel = None) -> dict:
    # Create noisy simulator backend
    if noise:
        sim = AerSimulator(noise_model=noise)
    else:
        sim = AerSimulator()
    
    # Transpile circuit for noisy basis gates
    passmanager = generate_preset_pass_manager(
        optimization_level=3, backend=sim
    )
    qc = passmanager.run(qc)

    # Run
    job = sim.run(qc, shots=shots)
    result = job.result()
    counts = result.get_counts()

    return counts

# Algorithms

## Random Number Generator

In [45]:
def qrng_sim_dist(coeffs: np.ndarray, shots: int) -> dict:    
    # Infer number of qubits
    numq = int(np.log2(len(coeffs)))

    # Normalize the coefficients
    coeffs /= np.sqrt(np.sum(np.square(coeffs)))

    # Initialize the circuit
    qc = QuantumCircuit(numq)

    # Prepare the generic quantum state which gives the desired distribution
    qc.initialize(list(coeffs), range(numq))

    # Measure all qubits
    qc.measure_all()

    # Run the circuit on a simulator backend
    sim = AerSimulator()

    # Run
    job = sim.run(qc, shots=shots)
    result = job.result()
    counts = result.get_counts()
        
    # Convert bin to dec
    counts_dec = {int(res, 2): freq for res, freq in counts.items()}

    return counts_dec

## Quantum State Tomography

In [ ]:
# This paper, section II B - https://arxiv.org/pdf/1407.4759
def inversion_physical(n0: list[int], n1: list[int]) -> np.ndarray:
    nx0, ny0, nz0 = n0
    nx1, ny1, nz1 = n1

    # Do the inversion
    rd = np.array([(nx0 - nx1) / (nx0 + nx1), (ny0 - ny1) / (ny0 + ny1), (nz0 - nz1) / (nz0 + nz1)])

    # We are only dealing with pure states, hence |Bloch vector| = 1
    rd = rd / np.linalg.norm(rd)
    
    return rd


# Same thing, but now allow for mixed states
def inversion(n0: list[int], n1: list[int]) -> np.ndarray:
    nx0, ny0, nz0 = n0
    nx1, ny1, nz1 = n1

    # Do the inversion
    rd = np.array([(nx0 - nx1) / (nx0 + nx1), (ny0 - ny1) / (ny0 + ny1), (nz0 - nz1) / (nz0 + nz1)])

    # We have that |Bloch vector| <= 1
    if (norm := np.linalg.norm(rd)) > 1:
        rd /= norm
    
    return rd

In [ ]:
def qst_state(instate: list, shots=1000) -> np.ndarray:
    
    # Create the QST circuit
    qc = QuantumCircuit(3)

    # Initialize all qubits in the same state
    for index in range(3):
        qc.initialize(instate, [index])

    # Measure the qubits in different basis (X, Y, Z basis respectively)
    qc.h(0)
    qc.sdg(1)
    qc.h(1)
    qc.id(2)

    qc.measure_all()


    # Simulate the circuit
    sim = AerSimulator()

    # Run
    job = sim.run(qc, shots=shots)
    result = job.result()
    counts = result.get_counts()

    # Parse the output according to direction
    n0 = [0, 0, 0]
    n1 = [0, 0, 0]
    for item in counts.items():
        string, freq = item
        string = string[::-1]
        for index, result in enumerate(string):
            if int(result) == 0:
                n0[index] += freq
            else:
                n1[index] += freq
    
    return inversion_physical(n0, n1)

In [ ]:
def qst_qubit(qc: QuantumCircuit, qubit: int, shots: int = 1024, noise: None | NoiseModel = None) -> np.ndarray:

    # Measure the qubit in different basis (X, Y, Z basis respectively)

    n0 = [0, 0, 0]
    n1 = [0, 0, 0]
    for basis in range(3):
        qccopy = qc.copy()
        if basis == 0:
            qccopy.h(qubit)
        elif basis == 1:
            qccopy.sdg(qubit)
            qccopy.h(qubit)
        elif basis == 2:
            qccopy.id(qubit)
        
        qccopy.measure_all()

        # Run
        counts = aersim(qccopy, shots, noise)

        # Parse the output according to direction
        for item in counts.items():
            string, freq = item
            string = string[::-1]
            result = string[qubit]
            if int(result) == 0:
                n0[basis] += freq
            else:
                n1[basis] += freq
    
    return inversion(n0, n1)

# Visualization

In [48]:
def plot_counts(counts: dict, bin_size: int, label: str = "Sample") -> tuple[np.ndarray | list[np.ndarray], np.ndarray]:
    # Parse the data
    outcomes = np.array(list(counts.keys()))
    freqs = np.array(list(counts.values()))
    
    # Make the plot
    bins = len(set(outcomes)) // bin_size

    n, bin_edges, _ = plt.hist(outcomes, bins=bins, edgecolor='black', weights=freqs / shots * 100, label=label)
    plt.xlabel("Outcome")
    plt.ylabel("Counts (%)")
    plt.title(f"Measurement Results (with {bins} bins)")
    plt.grid(axis='y', linestyle='--', alpha=0.5)

    return n, bin_edges